In [1]:
import os
import re
import torch
from transformers import AutoModel, AutoTokenizer
import gradio as gr
import mdtex2html
import platform
from utility.utils import config_dict
from utility.loggers import logger
from sentence_transformers import util
from local_database import db_operate
from utils import obtain_sql, retrieval_related_table, execute_sql
from prompt import query_template, chatbot_prompt

# Directly specify the device to use cuda:5
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("/data1/dxw_data/llm/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/data1/dxw_data/llm/chatglm3-6b", trust_remote_code=True).half().to(device)
model = model.eval()

def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    print(text)
    return text

def filter_input(input_text):
    """Function to filter out specified characters from the input text."""
    filtered_text = input_text.replace("你好", "")
    return filtered_text

def predict(input, history):
    max_length = 2048
    top_p = 0.7
    temperature = 0.2
    dboperate = db_operate(config_dict['db_path'])
    input_prompt = chatbot_prompt
    input_prompt = retrieval_related_table(input_prompt, input, history, top_k=3)
    input_prompt += query_template
    query = input_prompt.replace("<user_input>", input)
    
    response, history = model.chat(tokenizer, query, history=history, max_length=max_length, top_p=top_p, temperature=temperature)
    
    response = parse_text(response)
    response = obtain_sql(response)
    
    chatbot = [("", "")]  # Initialize chatbot list with an empty entry
    print(f"Debug: Initial chatbot: {chatbot}")  # Print initial chatbot list
    chatbot = execute_sql(response, chatbot, dboperate)
    
    return response, history

# Simulate user input and history
user_input = "你好，请帮我查询所有的事件类型"
history = []

# Apply the filter to the user input
filtered_input = filter_input(user_input)

# Call the predict function with the filtered input
response, history = predict(filtered_input, history)

# Print the response
print(response)


2024-07-27 16:48:41.558986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-27 16:48:41.717216: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-27 16:48:42.412253: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-07-27 16:48:42.412335: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

SELECT event_type FROM event_record
Debug: Initial chatbot: [('', '')]
Debug: Response in execute_sql: SELECT event_type FROM event_record
Debug: Chatbot before SQL execution: [('', '')]
Debug: Chatbot before updating: [('', '')]
Debug: Chatbot after updating: [('', "\n\n====================\n\nsql语句执行成功,结果如下:\n\n[('打架',), ('聚众',), ('翻越',), ('打架',), ('聚众',), ('翻越',), ('打架',), ('聚众',), ('翻越',)]")]
SELECT event_type FROM event_record
